# Downloading CMIP6 Experiment Data

This notebook steps through how to query and download the CMIP6 historic and projection datasets used later to build our ensemble projections. For a great rundown of what is going on in this notebook that I found *after* working on it, check this jupyter book out: [Earth and Environmental Data Science](https://earth-env-data-science.github.io/lectures/models/cmip.html)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
from dask.diagnostics import ProgressBar
from numpy import unique
import cf_xarray
import operator
import os
import sys
sys.path.append('Code/')
import fcts
import glob
import xesmf as xe
import dask
import xmip 

# XMIP preprocessing
from xmip.preprocessing import combined_preprocessing

# Function to help handle irregular grids
from xmip.preprocessing import replace_x_y_nominal_lat_lon

# Set Model Info/Criteria

This section sets up the configuration options including:
 * Model sources
 * Variable(s) of interest
 * Frequency of measurements
 * output path for saving subset data from CMIP6 models

These settings are used to query a catalog of available models and experiments.

In [3]:
# models we care about
# # These weren't working for Andrew, lets try again
source_list = [
    'IPSL-CM6A-LR', #
    'CMCC-CM2-SR5',
    'MIROC6', #
    'CanESM5',
    'MRI-ESM2-0',
    'HadGEM3-GC31-LL', # 
    'GFDL-CM4',
    'CESM2',
    'CESM2-WACCM',
    'CIESM', #
    'CNRM-CM6-1', #
    'CNRM-ESM2-1', #
    'CanESM5-CanOE',
    'EC-Earth3',
    'EC-Earth3-Veg', #
    'EC-Earth3-Veg-LR', #
    'FGOALS-g3',
    'FGOALS-f3-L',
    'FIO-ESM-2-0',
    'GISS-E2-1-G', #
    'INM-CM4-8',
    'INM-CM5-0'
    'MIROC-ES2L', #
    'NESM3', #
    'NorESM2-LM', #
    'NorESM2-MM', #
    'UKESM1-0-LL' #
]


# enter the table id (based on the frequency of measurements)
table_id = 'Omon'

# Enter the experiments of interest {'historical', 'ssp585', 'ssp126'}
filter_list    = ['ssp585']
experiment_run = 'ssp585'
grp1           = 'source_id' # used for grouping normally don't need to change
grp2           = 'member_id' # used for grouping normally don't need to change

# Output path
# path = "/Users/aallyn/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/"
path = "/Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/"


# Set some filtering options for x,y,z extents
# These were the original RawTmp Lims
mod_lims = {
    #"xmin" : -100,
    #"xmax" : -40,
    #"ymin" : 20,
    #"ymax" : 70,
    #"max_z" : 400
}

# These are the limits GLORYs was downloaded for crossboundary
mod_lims = {
    "xmin" : -100,
    "xmax" : -40,
    "ymin" : 20,
    "ymax" : 70,
    "max_z" : 400
}


# Enter the var of interest {so, thetao, tos}
# This spreadsheet has a key of all of them: (over 1000, sheeesh)
# https://docs.google.com/spreadsheets/u/1/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y/edit#gid=1221485271

# thetao = sea water potential temperature deg c
# so     = sea water salinity
# tos    = sea surface temperature deg C

# Set the variable to download
variable_id = 'thetao'

# True/False for bottom salinity or temperature from thetao or so variables
TOP = False 

## Query Model Catalog for Suitable Runs

Using those config options we can query the cmip6 consolidated stores catalog for the suite of models that fit the criteria we need. 

This catalog contains information on which institution ran the model, which scenario it used, and other information about the model. But most importantly, it provides a `ztore` field which has information on where to access the data.

Another avenue for accessing this caatalog is [intake-esm](https://intake-esm.readthedocs.io/en/stable/index.html) to load a collection of zarr stores on Google Cloud Storage. There are [other options](https://pangeo-data.github.io/pangeo-cmip6-cloud/accessing_data.html) to access the data too, but these are the ones I've seen so far.

In [4]:
# Data catalog is stored as a 30MB CSV file
# the columns correspond to the CMIP6 controlled vocab
AllModels = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

# To run/query a suite of models
# We can filter using the config options in the above chunk
df_var = AllModels.query(f"variable_id == '{variable_id}' & table_id == '{table_id}' & experiment_id == @filter_list")
filteredModels = fcts.ExperimentFilter(df_var, grp1, grp2)

# filter sources explicitly
filteredModels_grid = filteredModels.query(f"source_id == @source_list").reset_index(drop=True)

# look at it's structure
print(filteredModels_grid[0:3])

     source_id  member_id  Number_of_exp  activity_id institution_id  \
0        CESM2  r10i1p1f1              1  ScenarioMIP           NCAR   
1  CESM2-WACCM   r1i1p1f1              1  ScenarioMIP           NCAR   
2        CIESM   r1i1p1f1              1  ScenarioMIP            THU   

  experiment_id table_id variable_id grid_label  \
0        ssp585     Omon      thetao         gn   
1        ssp585     Omon      thetao         gr   
2        ssp585     Omon      thetao         gn   

                                              zstore  dcpp_init_year   version  
0  gs://cmip6/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585...             NaN  20200528  
1  gs://cmip6/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/...             NaN  20190815  
2  gs://cmip6/CMIP6/ScenarioMIP/THU/CIESM/ssp585/...             NaN  20200417  


# Iterate Over Model List and Extract + Save Data

From that list of suitable candidate models, we can open slice out the variable(s) we need for the xyz limits we're using. 

This chunk of code saves files in the location set by the `path` object stored earlier under the `RawTmpFiles/` subfolder. Bottom layer datasets are renamed for identification later. 

#### Why a separate loop for SO & Thetao?

Working with these variables involves dealing with the depth dimension, but otherwise its the same code as below for TOS (sea surface temperature.)

In [ ]:
# Adding preprocessing here?


In [5]:
# Salinity & Sea Water Potential
if variable_id in ["so", "thetao"]:

    # Only has to be defined once
    gcs = gcsfs.GCSFileSystem(token = 'anon')

    # Loop over each row in the query results
    for i in range(len(filteredModels_grid)):

        # Record information on the source, member, experiment, and variable
        source_id     = filteredModels_grid.source_id[i]
        member_id     = filteredModels_grid.member_id[i]
        experiment_id = filteredModels_grid.experiment_id[i]
        variable_id   = filteredModels_grid.variable_id[i]
        
        # Set export path(s)

        # Toggle for surface/bottom - sets filename convention for saving
        if TOP == True:
            savePath = f'{path}RawTmpFiles/Surface_{variable_id}_{source_id}_{member_id}_{experiment_id}.nc'
            # savePath = f'{path}RawTmpTest/Surface_{variable_id}_{source_id}_{member_id}_{experiment_id}.nc'
            
        else:
            savePath = f'{path}RawTmpFiles/{variable_id}_{source_id}_{member_id}_{experiment_id}.nc'
            #savePath = f'{path}RawTmpTest/{variable_id}_{source_id}_{member_id}_{experiment_id}.nc'
        
        # Skip if the download exists
        fileExists = os.path.exists(savePath)
        if fileExists:
            print(f"Skipping {savePath} as it already exists.")
            continue
        

        # Proceed to download if a copy not found
        if fileExists == False :
            # Progress update
            print(f"Working on {savePath}")
            
            # Get the path to a specific zarr store 0 index is first on list
            zstore = filteredModels_grid.zstore.values[i]

            # create a mutable-mapping-styly interface to the store (lazy-load it)
            mapper = gcs.get_mapper(zstore)

            # open it using xarray and zarr
            ds = xr.open_zarr(mapper, consolidated=True)

            # Pull the coordinates from CFDataAccessors
            lonNames = list(ds.cf[['longitude']].coords)
            latNames = list(ds.cf[['latitude']].coords)

            # Attempt the same with some common depth names
            try:
                vertNames = list(ds.cf[['vertical']].coords)
            except KeyError:
                vertNames = list(ds.cf[['Z']].coords)

            # Lists out the possible coordinate names that the model uses for them:
            lons  = ['lon', 'longitude', 'nav_lon', 'x']
            lats  = ['lat', 'latitude', 'nav_lat', 'y']
            verts = ['lev', 'olevel', "z"]

            # Are any names for the coordinates indexed?
            ds_coords = set(list(ds.cf[["latitude"]].coords))
            ds_indexes = set(list(ds.cf[["latitude"]].indexes))
            indexed_coords = ds_coords.intersection(ds_indexes)


            # If so, we can directly subset with those later instead of masking
            #if (len(indexed_coords) == 2):
            if (len(indexed_coords) == 2) & ("x" in indexed_coords):

                # Grab the xy coordinate names used in the dataset as a generic "x_coord" etc.
                x_coord = list(indexed_coords.intersection(lons))[0]
                y_coord = list(indexed_coords.intersection(lats))[0]
            
            # If not, don't worry about it and take the first one, 
            # only an issue for multiindex ones
            #else (len(indexed_coords) != 2) | ("nlon" in indexed_coords):
            else:
                
                # Grab the coordinate name used in the dataset as a generic "x_coord" etc.
                x_coord = list(set(lonNames).intersection(lons))[0]
                y_coord = list(set(latNames).intersection(lats))[0]
            

            # Determine whether its 0-360 or -180 to 180
            x_coordMin = ds[x_coord].values.min()
            
            # Adjust xlim cropping based on longitude coordinate system
            if x_coordMin < 0:
                xmin = mod_lims["xmin"]
                xmax = mod_lims["xmax"]
            else:
                xmin = mod_lims["xmin"] + 360
                xmax = mod_lims["xmax"] + 360

            # y lims
            ymin, ymax = mod_lims["ymin"], mod_lims["ymax"]

            # Get subset logic for slicing coordinates
            kwlon = {x_coord: slice(xmin, xmax)}
            kwlat = {y_coord: slice(ymin, ymax)}  



            # Check if it has Depth
            depth_coord = list(set(vertNames).intersection(verts))[0]

            # and if so what units
            try:
                levUnits = ds[depth_coord].units
            
            except AttributeError:
                print('No depth units')
                print(ds[depth_coord])
                print('Enter units')
                levUnits = input()

            # Set max depth - adjust for units
            if levUnits in ['m', 'meters']:
                maxDepth = mod_lims["max_z"]

            elif levUnits in ['cm', 'centimeters']:
                maxDepth = mod_lims["max_z"] * 100

            else:
                ds[depth_coord]
                print('Check attributes for depth information')


            # Check if its a multiIndex / irregular grid situation
            is_irregular = ds[y_coord].ndim == 2
            

            # Single index coordinate filter
            if is_irregular == False:
                atlantic = ds.sel(**kwlon, **kwlat)


            # MultiIndex / Irregular Grid Coordinate Filter
            if is_irregular == True:

                # Sometimes they just have x,y as indices... annoying
                # Subset it with ds.sel if they do
                if ["x"] in list(ds.cf[[y_coord]].indexes):    
                    atlantic = ds.sel(**kwlon, **kwlat)


                # If they don't, we do the hard way:
                else:
                    
                    # load coordinates into memory, use compute not values to preserve xarray array
                    print(f"Masking xylims with {x_coord} & {y_coord} coordinates")
                    lat = ds[y_coord].compute()
                    lon = ds[x_coord].compute()

                    # Create a mask for the desired range
                    mask = (lat >= ymin) & (lat <= ymax) & (lon >= xmin) & (lon <= xmax)
                    
                    # And subset with the mask
                    atlantic = ds.where(mask, drop=True)


            # Filter Dates for Historical or Not
            if experiment_id == 'historical':
                atlantic = atlantic.sel(time=slice("1950-01-01", None))
            
            # End at 2100
            # Need to update this experiment ID for each run
            elif experiment_id == experiment_run:  
                atlantic = atlantic.sel(time = slice(None, '2100-12-31'))
            
            # If neither ask to specify
            else:
                print("Need to enter date range")


            # Subset top layer if pulling surface
            if TOP == True:
                kwargs = {depth_coord: 0}
                ds = atlantic.isel(**kwargs)
                ds = ds.rename({depth_coord: 'surface'})


            # Pull up to max depth for bottom
            else:
                # Slice up to the max depth
                kwargs = {depth_coord: slice(0, maxDepth)}
                bottom_400 = atlantic.sel(**kwargs)
                temp_array = bottom_400[variable_id]

                # For irregular grid, get dims
                if is_irregular == True:
                    dims0 = bottom_400[y_coord].dims[0]
                    dims1 = bottom_400[y_coord].dims[1]
                else:
                    dims0 = y_coord
                    dims1 = x_coord

                # Get the bottom z index in space
                depth_indices = fcts.find_deepest_depth_indices_CMIP6(bottom_400, dims0, dims1, variable_id, y_coord, x_coord)
                ind = xr.DataArray(depth_indices, dims=[dims0, dims1])

                kwdepth = {depth_coord: ind}
                dsSel = temp_array.isel(**kwdepth)
                ds = dsSel.to_dataset()
                ds = ds.rename({depth_coord: 'bottom'})

            # Prepare to save
            delayed_obj = ds.to_netcdf(savePath, compute=False)

            # Use compute to bring everything over locally
            with ProgressBar():
                results = delayed_obj.compute()


            print(f'Finished {variable_id}_{source_id}_{member_id}_{experiment_id}.nc')


Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_CESM2_r10i1p1f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_CESM2-WACCM_r1i1p1f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_CMCC-CM2-SR5_r1i1p1f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_CanESM5_r10i1p1f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_CanESM5-CanOE_r1i1p2f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/thetao_EC-Earth3_r101i1p1f1_ssp585.nc as it already exists.
Skipping /Users/adamkemberling/Library/CloudStorage/Box-Box/RES_Data/CMIP6/

/Users/adamkemberling/Documents/Repositories/lobSDM/.pixi/envs/default/lib/python3.10/site-packages/xarray/coding/times.py:1001: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/adamkemberling/Documents/Repositories/lobSDM/.pixi/envs/default/lib/python3.10/site-packages/xarray/coding/times.py:1001: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/adamkemberling/Documents/Repositories/lobSDM/.pixi/envs/default/lib/python3.10/site-packages/xarray/core/indexing.py:516: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, re

[########################################] | 100% Completed | 319.95 s
Finished thetao_MRI-ESM2-0_r1i1p1f1_ssp585.nc


# TOS Loop

In [5]:

# Sea Surface Temperature
if variable_id == 'tos':

    # Iterate over
    for i in range(len(filteredModels_grid)):

        # Record information on the source, member, experiment, and variable
        source_id     = filteredModels_grid.source_id[i]
        member_id     = filteredModels_grid.member_id[i]
        experiment_id = filteredModels_grid.experiment_id[i]
        variable_id   = filteredModels_grid.variable_id[i]
        savePath      = f'{path}RawTmpFiles/{variable_id}_{source_id}_{member_id}_{experiment_id}.nc'
        
        # If file exists, skip to next one
        if os.path.exists(savePath):
            print(f"Skipping {savePath} as it already exists.")
            continue
        
        # If it isn't downloaded, work on that
        else :

            # get the path to a specific zarr store 0 index is first on list
            zstore = filteredModels_grid.zstore.values[i]

            # create a mutable-mapping-styly interface to the store
            mapper = gcs.get_mapper(zstore)

            # open it using xarray and zarr
            ds = xr.open_zarr(mapper, consolidated=True)

            # Pull the coordinates from CFDataAccessors
            lonNames = list(ds.cf[['longitude']].coords)
            latNames = list(ds.cf[['latitude']].coords)

            # Lists of possible coordinate names:
            lons = ['lon', 'longitude', 'nav_lon', 'x']
            lats = ['lat', 'latitude', 'nav_lat', 'y']


            # Are any names for the coordinates indexed?
            ds_coords = set(list(ds.cf[["latitude"]].coords))
            ds_indexes = set(list(ds.cf[["latitude"]].indexes))
            indexed_coords = ds_coords.intersection(ds_indexes)

            # Can't handle the unstructured grids when j,i indices are used
            if "j" in indexed_coords:
                print(f"Skipping {savePath} as ji index unstructured grid un-subsettable.")
                continue

            # If so, we can directly subset with those
            #if (len(indexed_coords) == 2):
            if (len(indexed_coords) == 2) & ("x" in indexed_coords):

                # Grab the xy coordinate names used in the dataset as a generic "x_coord" etc.
                x_coord = list(indexed_coords.intersection(lons))[0]
                y_coord = list(indexed_coords.intersection(lats))[0]
            
            # If not, don't worry about it and take the first one, 
            # only an issue for multiindex ones
            elif (len(indexed_coords) != 2) | ("nlon" in indexed_coords):
                
                # Grab the coordinate name used in the dataset as a generic "x_coord" etc.
                x_coord = list(set(lonNames).intersection(lons))[0]
                y_coord = list(set(latNames).intersection(lats))[0]
            
            
            # Determine whether its 0-360 or -180 to 180
            x_coordMin = ds[x_coord].values.min()
            
            # Adjust xlim cropping based on longitude coordinate system
            if x_coordMin < 0:
                xmin = mod_lims["xmin"]
                xmax = mod_lims["xmax"]
            else:
                xmin = mod_lims["xmin"] + 360
                xmax = mod_lims["xmax"] + 360

            # y lims
            ymin, ymax = mod_lims["ymin"], mod_lims["ymax"]

            # Get subset logic for slicing coordinates
            kwlon = {x_coord: slice(xmin, xmax)}
            kwlat = {y_coord: slice(ymin, ymax)}      

            # Check if its a multiIndex / irregular grid situation
            is_irregular = ds[y_coord].ndim == 2
            
            # Single index coordinate filter
            if is_irregular == False:
                # Get subset logic for slicing coordinates
                kwlon = {x_coord: slice(xmin, xmax)}
                kwlat = {y_coord: slice(ymin, ymax)}  
                atlantic = ds.sel(**kwlon, **kwlat)


            # MultiIndex / Irregular Grid Coordinate Filter
            if is_irregular == True:

                # Sometimes they just have x,y as indices... annoying
                if ["x"] in list(ds.cf[[y_coord]].indexes):
                    # Subset if they do
                    atlantic = ds.sel(**kwlon, **kwlat)

                # If they don't, we do the hard way:
                else:
                    print(f"Masking xylims with {x_coord} & {y_coord} coordinates")
                    # load coordinates into memory, use compute not values to preserve xarray array
                    lat = ds[y_coord].compute()
                    lon = ds[x_coord].compute()

                    # Create a mask for the desired range
                    mask = (lat >= ymin) & (lat <= ymax) & (lon >= xmin) & (lon <= xmax)
                    
                    # And subset with the mask
                    atlantic = ds.where(mask, drop=True)



            # Filter Dates for Historical or Not
            if experiment_id == 'historical':
                atlantic = atlantic.sel(time=slice("1950-01-01", None))
            
            # End at 2100
            # Need to update this experiment ID for each run
            elif experiment_id == experiment_run:  
                atlantic = atlantic.sel(time = slice(None, '2100-12-31'))
            
            # If neither ask to specify
            else:
                print("Need to enter date range")
            

            # Prepare to save
            delayed_obj = atlantic.to_netcdf(savePath, compute=False)

            # Use compute to finish
            with ProgressBar():
                results = delayed_obj.compute()


# # Log some basic info?
# names = {'name': [], 'minDate': [], 'maxData': [], 'length': []}
# ncTimes = pd.DataFrame(data = names)

---

# Debugging failed downloads:

Due to inconsistencies between the models, the code above will fail on occassion. These can be due to a number of problems including: trying to load too much data at a time with compute(), variable/index name differences, and multiIndex usage.

The three primary reasons the original code was failing was:
 - too much data (ds.compute()) to load without first subsetting
 - incorrect x,y dimension names for subsetting dimensions
 - multiIndex structure used for unstructured grids that used vertices (j,i) as coordinates


#### Presently, These are the model runs that are failing and their causes:

 * thetao_GFDL-CM4_r1i1p1f1_ssp585.nc <- Problem was loading with compute() at beginning

 * thetao_EC-CESM2_r1i1p1f1_ssp585.nc <- x is not a valid dimension

 * thetao_EC-Earth3-Veg_r1i1p1f1_ssp585.nc <- x is not a valid dimension

 * thetao_EC-Earth3-Veg-LR_r1i1p1f1_ssp585.nc <- x is not a valid dimension

 * thetao_HadGEM3-GC31-LL_r1i1p1f3_ssp585.nc <- x is not a valid dimension

 * thetao_MIROC6_r10i1p1f1_ssp585.nc <- No depth units

 * thetao_NESM3_r1i1p1f1_ssp585.nc <- 'x' is not a valid dimension

 * thetao_NorESM2-LM_r1i1p1f1_ssp585.nc <- if len(ds[variable_id][x_coord].dims) == 2: list index out of range

 * thetao_NorESM2-MM_r1i1p1f1_ssp585.nc <- 'x' is not a valid dimension

 * thetao_CMCC-CM2-SR5_r1i1p1f1_ssp585.nc <- operands could not be broadcast together with shapes (292, 362) (292, 362, 4) ()

They can be opened for inspection this way:


In [7]:
# Get the row/index of a single model:

# thetao_CESM2_r10i1p1f1_ssp585.nc

# Known values - uses nlon&nlat
variable_id   = "thetao"
source_id     = "CESM2"
member_id     = "r10i1p1f1"
experiment_id = "ssp585"

# # Known values - use j&i
# variable_id   = "thetao"
# source_id     = "NorESM2-MM"
# member_id     = "r1i1p1f1"
# experiment_id = "ssp585"

# # Which one just has x & y but was multiindex??, not multiindex
# variable_id   = "thetao"
# source_id     = "GFDL-CM4"
# member_id     = "r1i1p1f1"
# experiment_id = "ssp585"

# # Not enough indices from masking?
# variable_id   = "thetao"
# source_id     = "CNRM-CM6-1"
# member_id     = "r1i1p1f2"
# experiment_id = "ssp585"

# # i,j situation
# variable_id     = "thetao"
# source_id     = "CMCC-CM2-SR5"
# member_id     = "r1i1p1f1"
# experiment_id = "ssp585"


# Find the index by matching
df = filteredModels_grid
matching_index = df[(df['variable_id'] == variable_id) & (df['source_id'] == source_id) & (df['member_id'] == member_id) & (df['experiment_id'] == experiment_id)].index

# Check it
print("Matching index:", matching_index[0]) 
df[(df['variable_id'] == variable_id) & (df['source_id'] == source_id) & (df['member_id'] == member_id) & (df['experiment_id'] == experiment_id)]


Matching index: 0


,source_id,member_id,Number_of_exp,activity_id,institution_id,experiment_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CESM2,r10i1p1f1,1,ScenarioMIP,NCAR,ssp585,Omon,thetao,gn,gs://cmip6/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585...,NaN,20200528


In [10]:

ds["longitude"].values

array([[72.5       , 73.5       , 74.5       , ..., 71.5       ,
        72.5       , 73.5       ],
       [72.5       , 73.5       , 74.5       , ..., 71.5       ,
        72.5       , 73.5       ],
       [72.5       , 73.5       , 74.5       , ..., 71.5       ,
        72.5       , 73.5       ],
       ...,
       [72.95764923, 73.04235077, 73.1153183 , ..., 72.8846817 ,
        72.95764923, 73.04235077],
       [72.989151  , 73.010849  , 73.03703308, ..., 72.96296692,
        72.989151  , 73.010849  ],
       [73.        , 72.989151  , 72.96296692, ..., 73.03703308,
        73.        , 72.989151  ]])

## Debugging failed download behavior

Use those indices to open them for inspection to debug.

In [7]:
# Only has to be defined once
gcs = gcsfs.GCSFileSystem(token = 'anon')

# get the path to a specific zarr store 0 index is first on list
# create a mutable-mapping-styly interface to the store
zstore = filteredModels_grid.zstore.values[matching_index[0]]
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated = True, chunks={"time": 10})
ds

/Users/adamkemberling/Documents/Repositories/lobSDM/.pixi/envs/default/lib/python3.10/site-packages/xarray/backends/api.py:365: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 10. This could degrade performance. Instead, consider rechunking after loading.
  var_chunks = _get_chunk(var, chunks, chunkmanager)


<xarray.Dataset> Size: 30GB
Dimensions:    (nlat: 384, nlon: 320, vertices: 4, lev: 60, d2: 2, time: 1032)
Coordinates:
    lat        (nlat, nlon) float64 983kB dask.array<chunksize=(384, 320), meta=np.ndarray>
    lat_bnds   (nlat, nlon, vertices) float32 2MB dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
  * lev        (lev) float64 480B 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    lev_bnds   (lev, d2) float32 480B dask.array<chunksize=(60, 2), meta=np.ndarray>
    lon        (nlat, nlon) float64 983kB dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon_bnds   (nlat, nlon, vertices) float32 2MB dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
  * nlat       (nlat) int32 2kB 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
  * nlon       (nlon) int32 1kB 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
  * time       (time) object 8kB 2015-01-15 13:00:00.000007 ... 2100-12-15 12...
    time_bnds  (time, d2) object 17kB dask.array<chunksize=(10, 2), meta=np.ndarray>
Dimensions without coordinates: vertices, d2
Data variables:
    thetao     (time, lev, nlat, nlon) float32 30GB dask.array<chunksize=(10, 60, 384, 320), meta=np.ndarray>
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   735110.0
    branch_time_in_parent:  735110.0
    case_id:                1733
    ...                     ...
    tracking_id:            hdl:21.14100/a249e211-1f7a-40b4-be45-f17156102211...
    variable_id:            thetao
    variant_info:           CMIP6 SSP5-8.5 experiments (2015-2100) with CAM6,...
    variant_label:          r10i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/a249e211-1f7a-40b4-be45-f17156102211...
    version_id:             v20200528

## Using CFDataAccessors to Flexibly Find Relevant Index/Coordinate

CFDataAccessors provide some standard naming conventions to access information by common names. These can be leveraged to identify the coordinate names used for latitude and longitude:

In [9]:
# What are CFDatasetAccessors?
# Well you can check them this way:
#ds.cf?

# How we can get the latitude coordinates
list(ds.cf[['latitude']].coords)

['latitude', 'vertices_latitude', 'i', 'j']

In [10]:
# We can get the indexing if the coords aren't themselves indexed
list(ds.cf[['latitude']].indexes)

['i', 'j']

In [15]:
# Pull the coordinates from CFDataAccessors
lonNames = list(ds.cf[['longitude']].coords)
latNames = list(ds.cf[['latitude']].coords)

# Lists out the possible coordinate names that the model uses for them:
lons  = ['lon', 'longitude', 'nav_lon', 'x']
lats  = ['lat', 'latitude', 'nav_lat', 'y']

# Are any names for the coordinates indexed?
ds_coords = set(list(ds.cf[["latitude"]].coords))
ds_indexes = set(list(ds.cf[["latitude"]].indexes))
indexed_coords = ds_coords.intersection(ds_indexes)


# If so, we can directly subset with those if they are x and y, i & j
if (len(indexed_coords) == 2) & ("j" in indexed_coords) == False:

    # Grab the xy coordinate names used in the dataset as a generic "x_coord" etc.
    x_coord = list(indexed_coords.intersection(lons))
    y_coord = list(indexed_coords.intersection(lats))

elif (len(indexed_coords) != 2) | ("j" in indexed_coords):
    # Grab the coordinate name used in the dataset as a generic "x_coord" etc.
    x_coord = list(set(lonNames).intersection(lons))[0]
    y_coord = list(set(latNames).intersection(lats))[0]

x_coord

'longitude'

## Dealing with Irregular Grid BS

Some of these models use an irregular grid and can't be directly cropped using latitude/longitude coordinates.

In [18]:
# If we can't use ds.where() without ds.compute()
# Can we load just the coordinates and do the mask on them?
# for multi index

# # Is this the same as ds.isel(idxwargs)?
# No. Is it better?

# atlantic = ds.where(
#     (xmin < ds[x_coord]) & 
#     (ds[x_coord] < xmax) & 
#     (20 < ds[y_coord]) & 
#     (ds[y_coord] < 70),
#     drop=True)

# Load just the coordinates*

# load lat/lon coordinates into memory
lat = ds[y_coord].values
lon = ds[x_coord].values

# Create a mask for the desired range - could be breaking here
mask = (lat >= mod_lims["ymin"]) & (lat <= mod_lims["ymax"]) & (lon >= mod_lims["xmin"]) & (lon <= mod_lims["xmax"])

# # Find the indices of the grid points that work
# indices = np.where(mask)
atlantic = ds.where(mask)
atlantic

ValueError: operands could not be broadcast together with shapes (292, 362) (292, 362, 4) ()

In [25]:
# Sometimes they just have x,y as indices... annoying
if ["x", "y"] in list(ds.cf[[y_coord]].indexes):
    
    # Determine whether its 0-360 or -180 to 180
    # This is a double check in case x,y use something different
    x_indexMin = ds.indexes["x"].values.min()

    # Adjust xlim cropping based on longitude coordinate system
    if x_indexMin < 0:
        xmin, xmax = mod_lims["xmin"], mod_lims["xmax"]
    else:
        xmin, xmax = mod_lims["xmin"] + 360, mod_lims["xmax"] + 360
    
    # Can be passed into ds.sel() for cropping
    kwx = {"x": slice(xmin, xmax)}
    kwy = {"y": slice(ymin, ymax)}
    
    atlantic = ds.sel(**kwx, **kwy )

atlantic

<xarray.Dataset> Size: 11GB
Dimensions:    (bnds: 2, y: 310, x: 237, vertex: 4, lev: 35, time: 1032)
Coordinates:
  * bnds       (bnds) float64 16B 1.0 2.0
    lat        (y, x) float32 294kB dask.array<chunksize=(310, 237), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 1MB dask.array<chunksize=(310, 237, 4), meta=np.ndarray>
  * lev        (lev) float64 280B 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 560B dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 294kB dask.array<chunksize=(310, 237), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 1MB dask.array<chunksize=(310, 237, 4), meta=np.ndarray>
  * time       (time) object 8kB 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
    time_bnds  (time, bnds) object 17kB dask.array<chunksize=(10, 2), meta=np.ndarray>
  * x          (x) float64 2kB -99.87 -99.62 -99.36 ... -40.65 -40.4 -40.15
  * y          (y) float64 2kB 20.08 20.31 20.55 20.78 ... 69.73 69.84 69.95
Dimensions without coordinates: vertex
Data variables:
    thetao     (time, lev, y, x) float32 11GB dask.array<chunksize=(10, 35, 310, 237), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    variable_id:            thetao
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-08-14;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/2c4ae4a0-f3b8-464f-8100-941cec329403...
    version_id:             v20180701

For irregular grids we need to know what they use to index with and how they relate to lat/lon. I think we can use cfdataaccessors again:

In [ ]:
# # This way takes forever and returns too many indices.


# # MultiIndex / Irregular Grid Coordinate Filter
# if is_irregular == True:

#     # load lat/lon coordinates into memory
#     lat = ds[y_coord].values
#     lon = ds[x_coord].values

#     # Create a mask for the desired range - could be breaking here
#     mask = (lat >= ymin) & (lat <= ymax) & (lon >= xmin) & (lon <= xmax)

#     # Find the indices of the grid points that work
#     indices = np.where(mask)

#     # What are the indexes
#     x_idx, y_idx = list(ds.cf[[y_coord]].indexes)

#     # Get the index subsetting for those
#     idxwargs = {
#         y_idx: indices[0],
#         x_idx: indices[1]}
#     atlantic = ds.isel(**idxwargs)

# # check it
# atlantic

<xarray.Dataset> Size: 791GB
Dimensions:    (i: 2527, j: 2527, lev: 30, bnds: 2, time: 1032)
Coordinates:
  * i          (i) int32 10kB 42 43 44 45 39 40 41 42 ... 68 69 70 71 72 73 74
  * j          (j) int32 10kB 8 8 8 8 9 9 9 9 9 9 ... 56 56 56 56 56 56 56 56 56
    latitude   (j, i) float64 51MB dask.array<chunksize=(218, 360), meta=np.ndarray>
  * lev        (lev) float64 240B 5.0 15.0 25.0 ... 4.538e+03 5.244e+03
    lev_bnds   (lev, bnds) float64 480B dask.array<chunksize=(30, 2), meta=np.ndarray>
    longitude  (j, i) float64 51MB dask.array<chunksize=(218, 360), meta=np.ndarray>
  * time       (time) object 8kB 2015-01-16 13:00:00.000007 ... 2100-12-16 12...
    time_bnds  (time, bnds) object 17kB dask.array<chunksize=(10, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    thetao     (time, lev, j, i) float32 791GB dask.array<chunksize=(10, 30, 218, 360), meta=np.ndarray>
Attributes: (12/51)
    Conventions:            CF-1.7 CMIP-6.2
    NCO:                    netCDF Operators version 4.8.1 (Homepage = http:/...
    activity_id:            ScenarioMIP
    branch_method:          Spin-up documentation
    branch_time_in_child:   735110.0
    branch_time_in_parent:  735110.0
    ...                     ...
    title:                  FGOALS-f3-L output prepared for CMIP6
    tracking_id:            hdl:21.14100/01c58f7e-55be-4698-a42f-f954eb1a90ca...
    variable_id:            thetao
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/01c58f7e-55be-4698-a42f-f954eb1a90ca...
    version_id:             v20191012

In [33]:
# Double check that we get the same dimensions out if we use the ones that sliced on x and y - looks good
if is_irregular == True:
    # For multi index structured grid - subset on xy limits
    atlantic = ds.sel(
        x = slice(xmin, xmax), 
        y = slice(mod_lims["ymin"], mod_lims["ymax"])
        # longitude = slice(xmin, xmax), 
        # latitude = slice(mod_lims["ymin"], mod_lims["ymax"])
    )
atlantic

KeyError: "'x' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'i': 360, 'j': 218, 'lev': 30, 'bnds': 2, 'time': 1032})"

In [ ]:
# These Give a different dimensional array?
                        # # Find the indices of the grid points that work
                        # indices = np.where(mask)

                        # # What are the indexes
                        # x_idx, y_idx = list(ds.cf[[y_coord]].indexes)

                        # # Get the index subsetting for those
                        # idxwargs = {
                        #     y_idx: indices[0],
                        #     x_idx: indices[1]}
                        
                        # # And subset
                        # atlantic = ds.isel(**idxwargs)